In [1]:
!/usr/bin/env python

import sys

PY3 = sys.version_info[0] == 3

if PY3:
    xrange = range

import numpy as np
import cv2
import matplotlib.pyplot as plt
import time, pickle #standard pythonic things. #picle is used to save objects to disk
import signal, datetime, time 

‘ЁбвҐ¬Ґ ­Ґ г¤ Ґвбп ­ ©вЁ гЄ § ­­л© Їгвм.


In [2]:
vc = cv2.VideoCapture(1) #0 for the first webcam, 1 for the second. This is V4L"

In [3]:
def angle_cos(p0, p1, p2):
    d1, d2 = (p0 - p1).astype ('float'), (p2 - p1).astype ('float')
    return abs (np.dot (d1, d2) / np.sqrt (np.dot (d1, d1) * np.dot (d2, d2)))

def signal_handler(signal, frame):
    # KeyboardInterrupt detected, exiting
    global is_interrupted
    is_interrupted = True

def imshow(frame, from_color_space='bgr'): #show a picture from webcam
    plt.figure()
    if from_color_space == 'bgr':
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    elif from_color_space == 'hsv':
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_HSV2RGB)    # makes the hsv image look real colored
    else:
        rgb_frame=frame
    return plt.imshow(rgb_frame)


def get_frame(): #get a picture from webcam
    timeout = time.time() + 0.1
    while time.time() < timeout:
        if vc.isOpened(): # try to get the first frame
            is_capturing, frame = vc.read()
       # else:
        #    raise (Exception("Unable to capture"))
    cv2.imwrite("photo.png".format(datetime.datetime.now()), frame)
    return frame

def find_squares(img):
    img = cv2.GaussianBlur (img, (5, 5), 0)
    hsv = cv2.cvtColor (img, cv2.COLOR_BGR2HSV)
    img = cv2.inRange (hsv, (5, 50, 50), (15, 255, 255))
    squares = []
    pts = []
    
    for gray in cv2.split (img):
        for thrs in xrange (0, 255, 26):
            if thrs == 0:
                bin = cv2.Canny (gray, 0, 50, apertureSize=5)
                bin = cv2.dilate (bin, None)
                cv2.imshow ('sq', bin)
            else:
                _retval, bin = cv2.threshold (gray, thrs, 255, cv2.THRESH_BINARY)
            bin, contours, _hierarchy = cv2.findContours (bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                #contours = cv2.findContours (bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)               
            for cnt in contours:
                cnt_len = cv2.arcLength (cnt, True)
                cnt = cv2.approxPolyDP (cnt, 0.02 * cnt_len, True)
                if len (cnt) == 4 and cv2.contourArea (cnt) > 400 and cv2.isContourConvex (cnt):
                    cnt = cnt.reshape (-1, 2)
                    max_cos = np.max ([angle_cos (cnt[i], cnt[(i + 1) % 4], cnt[(i + 2) % 4]) for i in xrange (4)])
                    if max_cos < 0.1:
                        squares.append (cnt)
    return squares

In [4]:
while True:
    #img = cv2.imread('photo.png')
    img = get_frame()
    squares = find_squares (img)
    cv2.drawContours (img, squares, -1, (0, 255, 0), 3)
    cv2.imshow ('sq', img)
    ch = cv2.waitKey (1)
if ch == 27:
    break
cv2.destroyAllWindows ()

KeyboardInterrupt: 

In [ ]:
cv2.destroyAllWindows ()